<a href="https://colab.research.google.com/github/bkimb12/Intex-Colab-File/blob/main/Finished_Regression_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#  THIS REGRESSION MODEL IS NOT IMPLEMENTED ON OUR WEBSITE. HOWEVER, IT PROVIDED VALUABLE INFORMATION AS WE MADE
#  DECISIONS REGARDING DATA USED, ALONG WITH THE IMPORTANT FEATURES TO INCLUDE AND CONSIDER IN THIS MODEL AND OTHERS.


# make sure to install these packages before running:
!pip install sodapy

import pandas as pd
from sodapy import Socrata
import requests
import json

In [ ]:
# More installs
from sqlalchemy import create_engine
import urllib.parse
urllib.parse.quote_plus("http://aa1zdq2gijc4ui.c1dtnhbcknoc.us-east-1.rds.amazonaws.com/")
!pip install mysql-connector-python

     |████████████████████████████████| 37.6 MB 1.5 MB/s 


In [ ]:
#In this cell, we include the functions that we might need to use as we try to run our model.

#Read Data
def get_data(records = 2000):
  import pandas as pd
  from sodapy import Socrata
  import requests
  import json

  # Unauthenticated client only works with public data sets. Note 'None'
  # in place of application token, and no username or password:
  client = Socrata("opendata.utah.gov", 'GkoHUxXZ1JrjaFB0FlLHpniwf')

  # First 2000 results, returned as JSON from API / converted to Python list of
  # dictionaries by sodapy.
  results = client.get("herb-zqda", limit = records)

  # Convert to pandas DataFrame
  df = pd.DataFrame.from_records(results)

  return df

# Handle missing data

def drop_columns_missing_data(df, percent = 0.5):
  for col in df:
    if (df[col].isna().sum() / len(df)) > percent:
      df.drop(columns = [col], inplace = True)

  return df

In [ ]:
#This will collect all of the data into a dataframe that we will alter below
df_all = get_data(252500)

In [ ]:
#First we set df = df_all. This helps us if we don't want to get the data from the API everytime, allowing for faster subsequent queries.
df = df_all

In [ ]:
from datetime import datetime

#This creates a new column, generated from crash_datetime that shows the time of day that the crash occurred
df['crash_time'] = pd.to_datetime(df['crash_datetime'], format='%Y-%m-%d %H:%M:%S.%f').dt.hour

#This creates a new column, generated from crash_datetime that shows the day of the month that the crash occurred
df['crash_day'] = pd.to_datetime(df['crash_datetime'], format='%Y-%m-%d %H:%M:%S.%f').dt.day

#This creates a new column, generated from crash_datetime that shows the day of the week that the crash occurred
df['crash_dayofweek'] = pd.to_datetime(df['crash_datetime'], format='%Y-%m-%d %H:%M:%S.%f').dt.dayofweek

#This creates a new column, generated from crash_datetime that shows the year that the crash occurred
df['crash_year'] = pd.to_datetime(df['crash_datetime'], format='%Y-%m-%d %H:%M:%S.%f').dt.year

#This creates a new column, generated from crash_datetime that shows the month that the crash occurred
df['crash_month'] = pd.to_datetime(df['crash_datetime'], format='%Y-%m-%d %H:%M:%S.%f').dt.month

In [ ]:
#Here we drop any columns that we won't be using as part of our classification model
df = df.drop(columns=['main_road_name','long_utm_x','lat_utm_y','milepoint','route', 'crash_id', 'crash_day', 'crash_datetime', 'city', 'county_name'])

In [ ]:
#The first function called drops any columns with more than 50% of it's data missing (this is not important needed here, since no columns are missing 
#that much data, but it could be useful with future datasets). The second function drops any rows in which the crash_severity_id (which will be the 
#label) is missing.

df = drop_columns_missing_data(df)
df = df.dropna(subset=['crash_severity_id'])
df = df.dropna(subset=['work_zone_related'])

In [ ]:
#This is helpful to show the distrubution of crashes in the dataset by varying levels of severity

df['crash_severity_id'].value_counts()

1    177189
2     43871
3     23889
4      4652
5       960
Name: crash_severity_id, dtype: int64

In [ ]:
#For any other variable that has missing values (in this case it is only the work_zone_related field), this guesses the variable for that column. It
#also creates dummy codes for the other non-numeric variables. This also changes crash severity id to a float, so we need to make sure that is changed back.
#This is done in the next cell down
#df = impute_mean(df, 'crash_severity_id')

for col in df:
  if not pd.api.types.is_numeric_dtype(df[col]) and col != 'crash_severity_id':
    df = pd.get_dummies(df, columns=[col], drop_first=True)

In [ ]:
#Regression Model
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt

df['crash_severity_id'] = df['crash_severity_id'].astype(int)

y = df['crash_severity_id']
X = df
#Here we drop the label and any additional columns with high p-values
X = X.drop(columns=['crash_severity_id','night_dark_condition_True','crash_dayofweek'])
model = sm.OLS(y, X)
results = model.fit()
print(results.summary())

                                 OLS Regression Results                                
Dep. Variable:      crash_severity_id   R-squared (uncentered):                   0.818
Model:                            OLS   Adj. R-squared (uncentered):              0.818
Method:                 Least Squares   F-statistic:                          5.372e+04
Date:                Fri, 08 Apr 2022   Prob (F-statistic):                        0.00
Time:                        19:01:14   Log-Likelihood:                     -2.6440e+05
No. Observations:              250561   AIC:                                  5.288e+05
Df Residuals:                  250540   BIC:                                  5.291e+05
Df Model:                          21                                                  
Covariance Type:            nonrobust                                                  
                                         coef    std err          t      P>|t|      [0.025      0.975]
-----------------